In [1]:
import json
import os
import pandas as pd
import numpy as np

In [4]:
%cd Documents/GitHub/pan-clef-2024/task

C:\Users\Nadia Timoleon\Documents\GitHub\pan-clef-2024\task


## Utils

In [6]:
def json_to_dataframe(json_file):
  with open(json_file, 'r') as f:
    data = json.load(f)
  return pd.DataFrame(data)

def convert_json_files_to_pickle(json_dir, pickle_dir):
  # Create pickle directory if it doesn't exist
  if not os.path.exists(pickle_dir):
    os.makedirs(pickle_dir)
  
  # Loop through each json file in the directory
  for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
      json_path = os.path.join(json_dir, filename)
      df = json_to_dataframe(json_path)
      pickle_path = os.path.join(pickle_dir, os.path.splitext(filename)[0] + '.pkl')
      df.to_pickle(pickle_path)
      print(f"Converted {filename} to {pickle_path}")

In [5]:
convert_json_files_to_pickle(data_path + 'json', data_path + 'pickle')

Converted dataset_en_split_dev.json to ./Documents/GitHub/pan-clef-2024/data/pickle\dataset_en_split_dev.pkl
Converted dataset_en_split_train.json to ./Documents/GitHub/pan-clef-2024/data/pickle\dataset_en_split_train.pkl
Converted dataset_en_train.json to ./Documents/GitHub/pan-clef-2024/data/pickle\dataset_en_train.pkl
Converted dataset_es_split_dev.json to ./Documents/GitHub/pan-clef-2024/data/pickle\dataset_es_split_dev.pkl
Converted dataset_es_split_train.json to ./Documents/GitHub/pan-clef-2024/data/pickle\dataset_es_split_train.pkl
Converted dataset_es_train.json to ./Documents/GitHub/pan-clef-2024/data/pickle\dataset_es_train.pkl


## Revert spaCy tokenization

In [12]:
# TODO

## Concatenate EN and ES data

In [10]:
df_en = json_to_dataframe('../data/dataset_en_train.json')
df_en['lang'] = 'EN'
df_en.head()

,id,text,category,annotations,spacy_tokens,lang
0,5206,THIS IS MASSIVE Australian Senator Malcolm Rob...,CONSPIRACY,[{'span_text': 'Australian Senator Malcolm Rob...,WyJUSElTIiwgIklTIiwgIk1BU1NJVkUiLCAiQXVzdHJhbG...,EN
1,1387,“ I ’m deeply concerned that the push to vacci...,CRITICAL,[{'span_text': 'I ’m deeply concerned that the...,WyJcdTIwMWMiLCAiSSIsICJcdTIwMTltIiwgImRlZXBseS...,EN
2,13116,2021 : They wanted to know your vaccination st...,CRITICAL,"[{'span_text': 'someone who died suddenly', 'c...",WyIyMDIxIiwgIjoiLCAiVGhleSIsICJ3YW50ZWQiLCAidG...,EN
3,11439,Anthony Fauci once again defended brutal Chine...,CRITICAL,"[{'span_text': 'brutal Chinese lockdowns', 'ca...",WyJBbnRob255IiwgIkZhdWNpIiwgIm9uY2UiLCAiYWdhaW...,EN
4,98,Proof has emerged showing that death from Wuha...,CRITICAL,[{'span_text': 'death from Wuhan coronavirus (...,WyJQcm9vZiIsICJoYXMiLCAiZW1lcmdlZCIsICJzaG93aW...,EN


In [7]:
df_es = json_to_dataframe('../data/dataset_es_train.json')
df_es['lang'] = 'ES'
df_es.head()

,id,text,category,annotations,spacy_tokens,lang
0,2807,Fallo en Matrix 08/02/2022 Hoy el señor Joan R...,CRITICAL,[{'span_text': 'el señor Joan Ramón Laporte Ro...,WyJGYWxsbyIsICJlbiIsICJNYXRyaXgiLCAiMDgvMDIvMj...,ES
1,3054,Siento ya tdas las vacunas vienen contaminadas...,CRITICAL,"[{'span_text': 'mi sobrina', 'category': 'VICT...",WyJTaWVudG8iLCAieWEiLCAidGRhcyIsICJsYXMiLCAidm...,ES
2,268,Veo que curiosamente te autoproclamados interl...,CONSPIRACY,"[{'span_text': 'todo el grupo', 'category': 'C...",WyJWZW8iLCAicXVlIiwgImN1cmlvc2FtZW50ZSIsICJ0ZS...,ES
3,2669,[ Documental ] Vacunas : Una inyección en la o...,CRITICAL,[{'span_text': '[ Documental ] Vacunas : Una i...,WyJbIiwgIkRvY3VtZW50YWwiLCAiXSIsICJWYWN1bmFzIi...,ES
4,3205,Una sugerencia para los que se han vacunado y ...,CONSPIRACY,[{'span_text': 'los que se han vacunado y no q...,WyJVbmEiLCAic3VnZXJlbmNpYSIsICJwYXJhIiwgImxvcy...,ES


In [20]:
df_en_es = pd.concat([df_en, df_es])
# rename id column to old_id
df_en_es.rename(columns={'id': 'old_id'}, inplace=True)
df_en_es.reset_index(inplace=True, drop=True)
# create id column thats the same as the index and move it to the front
df_en_es['id'] = df_en_es.index
cols = list(df_en_es.columns)
cols = [cols[-1]] + cols[:-1]
df_en_es = df_en_es[cols]
df_en_es.head()

,id,old_id,text,category,annotations,spacy_tokens,lang
0,0,5206,THIS IS MASSIVE Australian Senator Malcolm Rob...,CONSPIRACY,[{'span_text': 'Australian Senator Malcolm Rob...,WyJUSElTIiwgIklTIiwgIk1BU1NJVkUiLCAiQXVzdHJhbG...,EN
1,1,1387,“ I ’m deeply concerned that the push to vacci...,CRITICAL,[{'span_text': 'I ’m deeply concerned that the...,WyJcdTIwMWMiLCAiSSIsICJcdTIwMTltIiwgImRlZXBseS...,EN
2,2,13116,2021 : They wanted to know your vaccination st...,CRITICAL,"[{'span_text': 'someone who died suddenly', 'c...",WyIyMDIxIiwgIjoiLCAiVGhleSIsICJ3YW50ZWQiLCAidG...,EN
3,3,11439,Anthony Fauci once again defended brutal Chine...,CRITICAL,"[{'span_text': 'brutal Chinese lockdowns', 'ca...",WyJBbnRob255IiwgIkZhdWNpIiwgIm9uY2UiLCAiYWdhaW...,EN
4,4,98,Proof has emerged showing that death from Wuha...,CRITICAL,[{'span_text': 'death from Wuhan coronavirus (...,WyJQcm9vZiIsICJoYXMiLCAiZW1lcmdlZCIsICJzaG93aW...,EN


In [21]:
# Save as pickle
df_en_es.to_pickle('../data/pickle/dataset_combined_train.pkl')

In [22]:
# Save as json
df_en_es.to_json('../data/dataset_combined_train.json', orient='records')

## Concatenate EN and ES real and augmented data

In [12]:
df_comb = json_to_dataframe('../data/dataset_combined_train.json')
df_comb.drop(columns=['annotations', 'spacy_tokens'])
df_comb.head()

,id,old_id,text,category,annotations,spacy_tokens,lang
0,0,5206,THIS IS MASSIVE Australian Senator Malcolm Rob...,CONSPIRACY,[{'span_text': 'Australian Senator Malcolm Rob...,WyJUSElTIiwgIklTIiwgIk1BU1NJVkUiLCAiQXVzdHJhbG...,EN
1,1,1387,“ I ’m deeply concerned that the push to vacci...,CRITICAL,[{'span_text': 'I ’m deeply concerned that the...,WyJcdTIwMWMiLCAiSSIsICJcdTIwMTltIiwgImRlZXBseS...,EN
2,2,13116,2021 : They wanted to know your vaccination st...,CRITICAL,"[{'span_text': 'someone who died suddenly', 'c...",WyIyMDIxIiwgIjoiLCAiVGhleSIsICJ3YW50ZWQiLCAidG...,EN
3,3,11439,Anthony Fauci once again defended brutal Chine...,CRITICAL,"[{'span_text': 'brutal Chinese lockdowns', 'ca...",WyJBbnRob255IiwgIkZhdWNpIiwgIm9uY2UiLCAiYWdhaW...,EN
4,4,98,Proof has emerged showing that death from Wuha...,CRITICAL,[{'span_text': 'death from Wuhan coronavirus (...,WyJQcm9vZiIsICJoYXMiLCAiZW1lcmdlZCIsICJzaG93aW...,EN


In [15]:
df_en_tr = json_to_dataframe('../data/dataset_EN_translated.json')
df_en_tr['lang'] = 'EN'
df_en_tr.head()

,id,text,category,lang
0,4000,Judgement in Matrix 08/02/2022 Today Mr. Joan ...,CRITICAL,EN
1,4001,I feel the vaccines come contaminated my niece...,CRITICAL,EN
2,4002,I see that curiously self-proclaimed interlocu...,CONSPIRACY,EN
3,4003,[ Documentary ] Vaccines : An injection in the...,CRITICAL,EN
4,4004,A suggestion for those who have been vaccinate...,CONSPIRACY,EN


In [14]:
df_es_tr = json_to_dataframe('../data/dataset_ES_translated.json')
df_es_tr['lang'] = 'ES'
df_es_tr.head()

,id,text,category,lang
0,4000,ESTO ES MASIVO El senador australiano Malcolm ...,CONSPIRACY,ES
1,4001,“ Me preocupa profundamente que el impulso par...,CRITICAL,ES
2,4002,2021 : Querían saber su estado de vacunación y...,CRITICAL,ES
3,4003,Anthony Fauci una vez más defendió brutales bl...,CRITICAL,ES
4,4004,La prueba ha surgido mostrando que la muerte d...,CRITICAL,ES


In [17]:
df_en_es_tr = pd.concat([df_en_tr, df_es_tr])
# rename id column to old_id
df_en_es_tr.rename(columns={'id': 'old_id'}, inplace=True)
df_en_es_tr.reset_index(inplace=True, drop=True)
# create id column thats the same as the index and move it to the front
df_en_es_tr['id'] = df_en_es_tr.index
cols = list(df_en_es_tr.columns)
cols = [cols[-1]] + cols[:-1]
df_en_es_tr = df_en_es_tr[cols]
df_en_es_tr.head()

,id,old_id,text,category,lang
0,0,4000,Judgement in Matrix 08/02/2022 Today Mr. Joan ...,CRITICAL,EN
1,1,4001,I feel the vaccines come contaminated my niece...,CRITICAL,EN
2,2,4002,I see that curiously self-proclaimed interlocu...,CONSPIRACY,EN
3,3,4003,[ Documentary ] Vaccines : An injection in the...,CRITICAL,EN
4,4,4004,A suggestion for those who have been vaccinate...,CONSPIRACY,EN


In [21]:
# Save as pickle
df_en_es_tr.to_pickle('../data/pickle/dataset_combined_plus_translated.pkl')

In [22]:
# Save as json
df_en_es_tr.to_json('../data/dataset_combined_plus_translated.json', orient='records')